# Colobrative Filtering -Using Surprise Libraray(KNNBasic Algo)

In [5]:
pip install surprise

     ---------------------------------------- 0.0/154.4 kB ? eta -:--:--
     -- ------------------------------------- 10.2/154.4 kB ? eta -:--:--
     --------- --------------------------- 41.0/154.4 kB 495.5 kB/s eta 0:00:01
     ------------ ------------------------ 51.2/154.4 kB 440.4 kB/s eta 0:00:01
     ------------------------------- ---- 133.1/154.4 kB 877.7 kB/s eta 0:00:01
     ------------------------------------ 154.4/154.4 kB 838.3 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-win_amd64.whl size=1289509 sha256=9dde7c919a03fee4d7767f445e8369a8d127a2ddaa6c7845c5f3c4aa9ea547e7
  Stored in directory: c

In [7]:
from surprise import Dataset,Reader,KNNBasic 
from surprise.model_selection import train_test_split
from surprise import accuracy
# Dataset- used to load builtin datasets
# Reader - used for custom datasets
# KNNBasic-Collabortive filtering algo based on KNN 


In [29]:
import pandas as pd

In [66]:
from surprise import get_dataset_dir
import os
print(os.listdir(get_dataset_dir()))

['ml-100k']


In [149]:
trainset,testset=train_test_split(data,test_size=0.25)
sim_options={
    'user':'msd',
    'user_based':False,
}

In [150]:
data= Dataset.load_builtin('ml-100k').raw_ratings
df=pd.DataFrame(data,columns=['user-id','movie_id','Ratings','TimeStamp'])
df.info()
print(df.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user-id    100000 non-null  object 
 1   movie_id   100000 non-null  object 
 2   Ratings    100000 non-null  float64
 3   TimeStamp  100000 non-null  object 
dtypes: float64(1), object(3)
memory usage: 3.1+ MB
      user-id movie_id  Ratings  TimeStamp
99995     880      476      3.0  880175444
99996     716      204      5.0  879795543
99997     276     1090      1.0  874795795
99998      13      225      2.0  882399156
99999      12      203      3.0  879959583


In [151]:
model=KNNBasic(sim_options=sim_options)
model.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


## Prediction of the movie based on similarity 

In [152]:
predict1=model.predict(uid='12',iid='203')
print(f"Predicted ratings for user {predict1.uid} on item {predict1.iid}: {predict1.est:.2f}")
predict1
# r_ui-true user rating given by the user, Not None if given manually

Predicted ratings for user 12 on item 203: 4.46


Prediction(uid='12', iid='203', r_ui=None, est=4.463028874348179, details={'actual_k': 36, 'was_impossible': False})

In [153]:
predict=model.test(testset)

In [154]:
print("RSME:",accuracy.rmse(predict))

RMSE: 0.9794
RSME: 0.9794376817450111


## KNNBasic to Item-Based Filtering

In [155]:
from collections import defaultdict
def get_top_n(predictions,n=5):
    top_n=defaultdict(list)
    # true_r->true ratings
    # est-> predicted rating
    for uid,iid,true_r,est,_ in predictions:
        top_n[uid].append((iid,est))
    for uid,user_ratings in top_n.items():
        user_ratings.sort(key=lambda x:x[1], reverse=True)
        top_n[uid]=user_ratings[:n]
    return top_n
    

In [156]:
test_predict=model.test(testset)

In [157]:
top_n=get_top_n(test_predict,n=5)
top_n

defaultdict(list,
            {'409': [('528', 4.069063990904951),
              ('134', 4.059957073744493),
              ('48', 4.02041422986584),
              ('136', 3.9889051865760745),
              ('427', 3.9762174516640982)],
             '942': [('269', 4.51154420399958),
              ('478', 4.481669664211675),
              ('487', 4.477840380420697),
              ('496', 4.467310791821836),
              ('607', 4.462479870283209)],
             '417': [('474', 3.8513629668802314),
              ('178', 3.839506628987868),
              ('168', 3.826394582887615),
              ('12', 3.788301922231299),
              ('69', 3.777978704114802)],
             '186': [('159', 3.680744062302956),
              ('550', 3.6521056295724),
              ('754', 3.648883932194228),
              ('331', 3.594871601075734),
              ('332', 3.58623944993501)],
             '250': [('485', 4.195122746409037),
              ('179', 4.165090210327957),
              ('9', 4.14

In [158]:
print(f"Top 5 Recommendations for user 196:")
for iid,rating in top_n['196']:
    print(f"Movie{iid}(Predicted rating:{rating:.2f})")

Top 5 Recommendations for user 196:
Movie1022(Predicted rating:3.72)
Movie285(Predicted rating:3.69)
Movie94(Predicted rating:3.67)
Movie393(Predicted rating:3.66)
Movie692(Predicted rating:3.63)


# defaultdict

In [159]:
from collections import defaultdict
d=defaultdict(lambda:"Empty String")
d['a']='hello'
d['b']='bye'
print(d['c'])
print(d.__missing__('a'))
#__missing__() function to show missing values
print(d.__missing__('b'))

Empty String
Empty String
Empty String


In [160]:
l=[(12,3.9),(14,4.9),(15,16.2)]
for uid,ratings in l:
    l.sort(key=lambda x:x[1],reverse=True)
print(l)

[(15, 16.2), (14, 4.9), (12, 3.9)]


#https://chat.deepseek.com/a/chat/s/a5a23195-4606-4675-92ea-be97c832f1ea